# Data Preparation and EDA for IoT23 dataset

In [4]:
import os 
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
import csv
from numpy import array
from numpy import argmax
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler,LabelEncoder

In [5]:
data= pd.read_csv('preprocessed_iot23.csv', low_memory=False)

In [6]:
data.head()  

,Unnamed: 0,ts,id.orig_p,id.resp_p,duration,orig_bytes,resp_bytes,local_orig,local_resp,missed_bytes,...,conn_state_RSTR,conn_state_RSTRH,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR,label
0,0,1.536227e+09,17576.0,8081.0,0.000003,0.0,0.0,0,0,0.0,...,0,0,1,0,0,0,0,0,0,POHScan
1,1,1.536227e+09,17576.0,8081.0,0.000002,0.0,0.0,0,0,0.0,...,0,0,1,0,0,0,0,0,0,POHScan
2,2,1.536227e+09,17576.0,8081.0,0.000002,0.0,0.0,0,0,0.0,...,0,0,1,0,0,0,0,0,0,POHScan
3,3,1.536227e+09,17576.0,8081.0,0.000002,0.0,0.0,0,0,0.0,...,0,0,1,0,0,0,0,0,0,POHScan
4,4,1.536227e+09,17576.0,8081.0,0.000002,0.0,0.0,0,0,0.0,...,0,0,1,0,0,0,0,0,0,POHScan


In [7]:
del data['Unnamed: 0']

In [8]:
data.tail()

,ts,id.orig_p,id.resp_p,duration,orig_bytes,resp_bytes,local_orig,local_resp,missed_bytes,orig_pkts,...,conn_state_RSTR,conn_state_RSTRH,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR,label
2802750,1.562165e+09,68.0,67.0,90.034713,3300.0,0.0,0,0,0.0,11.0,...,0,0,1,0,0,0,0,0,0,Benign
2802751,1.562165e+09,143.0,0.0,90.399970,340.0,0.0,0,0,0.0,9.0,...,0,0,0,0,0,0,0,0,0,Benign
2802752,1.562165e+09,135.0,136.0,89.824030,72.0,0.0,0,0,0.0,3.0,...,0,0,0,0,0,0,0,0,0,Benign
2802753,1.562165e+09,143.0,0.0,45.215915,200.0,0.0,0,0,0.0,8.0,...,0,0,0,0,0,0,0,0,0,Benign
2802754,1.562165e+09,133.0,134.0,44.242223,0.0,0.0,0,0,0.0,2.0,...,0,0,0,0,0,0,0,0,0,Benign


In [9]:
data.shape

(2802755, 30)

In [10]:
print(data.columns)

Index(['ts', 'id.orig_p', 'id.resp_p', 'duration', 'orig_bytes', 'resp_bytes',
       'local_orig', 'local_resp', 'missed_bytes', 'orig_pkts',
       'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp',
       'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ',
       'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR',
       'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2',
       'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR',
       'label'],
      dtype='object')


In [11]:
#separate the X variables
X=data.iloc[:,:-1]

In [12]:
#separate labels (y)
y=data['label']

In [13]:
X.describe()

,ts,id.orig_p,id.resp_p,duration,orig_bytes,resp_bytes,local_orig,local_resp,missed_bytes,orig_pkts,...,conn_state_RSTOS0,conn_state_RSTR,conn_state_RSTRH,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR
count,2.802755e+06,2.802755e+06,2.802755e+06,2.802755e+06,2.802755e+06,2.802755e+06,2802755.0,2802755.0,2.802755e+06,2.802755e+06,...,2.802755e+06,2.802755e+06,2.802755e+06,2.802755e+06,2.802755e+06,2.802755e+06,2.802755e+06,2.802755e+06,2.802755e+06,2.802755e+06
mean,1.539080e+09,3.733966e+04,2.294199e+04,3.764077e-01,1.509650e+03,1.283293e+02,0.0,0.0,1.349422e-02,1.819718e+02,...,7.028798e-05,5.444643e-04,2.426184e-05,8.410621e-01,1.213092e-05,1.462846e-05,9.633379e-06,7.802323e-03,4.781010e-05,4.638293e-06
std,1.440108e+07,1.465224e+04,2.397775e+04,6.546523e+01,9.245166e+05,2.011450e+05,0.0,0.0,6.924522e+00,7.340439e+04,...,8.383500e-03,2.332741e-02,4.925572e-03,3.656182e-01,3.482927e-03,3.824690e-03,3.103754e-03,8.798551e-02,6.914321e-03,2.153665e-03
min,1.525880e+09,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.526150e+09,3.314600e+04,2.300000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.532529e+09,3.795300e+04,9.527000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.545398e+09,4.549700e+04,3.721500e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,1.569018e+09,6.553500e+04,6.553500e+04,7.884033e+04,9.623472e+08,3.365164e+08,0.0,0.0,7.363000e+03,6.602735e+07,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [14]:
X.isnull().sum()

ts                   0
id.orig_p            0
id.resp_p            0
duration             0
orig_bytes           0
resp_bytes           0
local_orig           0
local_resp           0
missed_bytes         0
orig_pkts            0
orig_ip_bytes        0
resp_pkts            0
resp_ip_bytes        0
proto_icmp           0
proto_tcp            0
proto_udp            0
conn_state_OTH       0
conn_state_REJ       0
conn_state_RSTO      0
conn_state_RSTOS0    0
conn_state_RSTR      0
conn_state_RSTRH     0
conn_state_S0        0
conn_state_S1        0
conn_state_S2        0
conn_state_S3        0
conn_state_SF        0
conn_state_SH        0
conn_state_SHR       0
dtype: int64

In [15]:
X = np.asarray(X).astype(np.float32)

In [16]:
y

0          POHScan
1          POHScan
2          POHScan
3          POHScan
4          POHScan
            ...   
2802750     Benign
2802751     Benign
2802752     Benign
2802753     Benign
2802754     Benign
Name: label, Length: 2802755, dtype: object

In [17]:
y.value_counts()

POHScan    1324265
Okiru       549228
Benign      472319
DDoS        448703
C&C           8240
Name: label, dtype: int64

In [18]:
le=LabelEncoder()

In [19]:
y=le.fit_transform(y)

In [20]:
y

array([4, 4, 4, ..., 0, 0, 0])

In [21]:
#the labels are encoded in alphabetic order
#0---->Benign
#1---->Cnc
#2---->DDoS
#3---->Okiru
#4---->POHScan


In [22]:
le.inverse_transform(y)

array(['POHScan', 'POHScan', 'POHScan', ..., 'Benign', 'Benign', 'Benign'],
      dtype=object)

In [23]:
y.shape

(2802755,)

In [24]:
sc=StandardScaler()

In [25]:
Xs=sc.fit_transform(X)

=======================================================================

In [26]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn import neighbors
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import roc_auc_score
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import auc,roc_auc_score,roc_curve,precision_score,recall_score,f1_score
from sklearn.model_selection import train_test_split
import time as timer

In [27]:
X_train, X_test, Y_train, Y_test = train_test_split(Xs, y, random_state=124, test_size=0.30, shuffle=True)

# Gussian Naive Bayes 

In [61]:
GaussianNB?

In [56]:
start = timer.time()
gnb_model = GaussianNB()
gnb_model.fit(X_train, Y_train)
end = timer.time()
print("Finished training within {:.2f} seconds".format(end-start))

Finished training within 0.59 seconds


In [57]:
# Predicting the results on train set
y_gnbr = gnb_model.predict(X_train)

In [58]:
print("Accuracy score for GNB: {:.4f}".format(accuracy_score(Y_train,y_gnbr)))

Accuracy score for GNB: 0.6957


In [59]:
# Predicting the test set results
y_gnb = gnb_model.predict(X_test)
y_gnb_prob = gnb_model.predict_proba(X_test)

In [60]:
print("Accuracy score for GNB: {:.4f}".format(accuracy_score(Y_train,y_gnbr)))

Accuracy score for GNB: 0.6957


In [65]:
print("Classification report for GNB: \n{}".format(classification_report(Y_test,y_gnb)))
print("Confusion matrix for GNB: \n{}".format(confusion_matrix(Y_test,y_gnb)))
print("Accuracy score for GNB: {:.2f}".format(accuracy_score(Y_test,y_gnb)))
# calculate precision, recall, and f1 scores
prec_gnb = precision_score(Y_test,y_gnb,average='weighted')
rec_gnb = recall_score(Y_test,y_gnb,average='weighted')
f1_gnb = f1_score(Y_test,y_gnb,average='weighted')
print("Precision score for GNB: {:.2f}".format(prec_gnb))
print("Recall score for GNB: {:.2f}".format(rec_gnb))
print("F1 score for GNB: {:.2f}".format(f1_gnb))

Classification report for GNB: 
              precision    recall  f1-score   support

           0       0.98      0.91      0.94    236110
           1       0.01      1.00      0.02      4123
           2       0.95      1.00      0.97    224498
           3       1.00      1.00      1.00    274797
           4       0.97      0.39      0.56    661850

    accuracy                           0.70   1401378
   macro avg       0.78      0.86      0.70   1401378
weighted avg       0.97      0.70      0.77   1401378

Confusion matrix for GNB: 
[[214569    465  12034    134   8908]
 [     6   4116      0      0      1]
 [     2      0 224486      0     10]
 [     2      0      2 274734     59]
 [  3709 399881      0      0 258260]]
Accuracy score for GNB: 0.70
Precision score for GNB: 0.97
Recall score for GNB: 0.70
F1 score for GNB: 0.77


# Logistic Regression 

In [205]:
LogisticRegression?

In [66]:
start = timer.time()
logit_model = LogisticRegression(solver='sag', max_iter=300, multi_class='multinomial')
logit_model.fit(X_train, Y_train)
end = timer.time()
print("Finished training within {:.2f} seconds".format(end-start))

Finished training within 77.93 seconds


In [67]:
# Predicting the train set results
y_logitr = logit_model.predict(X_train)

In [71]:
print("Accuracy score for LR: {:.4f}".format(accuracy_score(Y_train,y_logitr)))

Accuracy score for LR: 0.9636


In [69]:
# Predicting the test set results
y_logit = logit_model.predict(X_test)
y_logit_prob = logit_model.predict_proba(X_test)

In [72]:
print("Classification report for LR: \n{}".format(classification_report(Y_test,y_logit)))
print("Confusion matrix for LR: \n{}".format(confusion_matrix(Y_test,y_logit)))
print("Accuracy score for LR: {:.4f}".format(accuracy_score(Y_test,y_logit)))
# calculate precision, recall, and f1 scores
prec_logit = precision_score(Y_test,y_logit,average='weighted')
rec_logit = recall_score(Y_test,y_logit,average='weighted')
f1_logit = f1_score(Y_test,y_logit,average='weighted')
print("Precision score for LR: {:.4f}".format(prec_logit))
print("Recall score for LR: {:.4f}".format(rec_logit))
print("F1 score for LR: {:.4f}".format(f1_logit))


C:\Users\DELL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification report for LR: 
              precision    recall  f1-score   support

           0       1.00      0.91      0.95    236110
           1       0.00      0.00      0.00      4123
           2       1.00      0.89      0.94    224498
           3       0.90      1.00      0.94    274797
           4       0.97      1.00      0.99    661850

    accuracy                           0.96   1401378
   macro avg       0.77      0.76      0.76   1401378
weighted avg       0.96      0.96      0.96   1401378

Confusion matrix for LR: 
[[214420      0    139   7082  14469]
 [     4      0      0      0   4119]
 [     7      0 199495  24996      0]
 [     1      0     59 274736      1]
 [    24      0      0      0 661826]]
Accuracy score for LR: 0.9637


C:\Users\DELL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision score for LR: 0.9635
Recall score for LR: 0.9637
F1 score for LR: 0.9620


# RandomForestClassifier

In [73]:
start = timer.time()
Rf_clf=RandomForestClassifier(n_estimators =10, criterion = 'entropy', random_state = 124)
Rf_clf.fit(X_train, Y_train)
end = timer.time()
print("Finished training within {:.4f} seconds".format(end-start))

Finished training within 24.6085 seconds


In [75]:
#checking train accuracy
y_RF_train = Rf_clf.predict(X_train)

In [76]:
print("Accuracy score for RF: {:.4f}".format(accuracy_score(Y_train,y_RF_train)))

Accuracy score for RF: 1.0000


In [77]:
y_RF = Rf_clf.predict(X_test)
y_RF_prob = Rf_clf.predict_proba(X_test)

In [78]:
print("Classification report for RF: \n{}".format(classification_report(Y_test,y_RF)))
print("Confusion matrix for RF: \n{}".format(confusion_matrix(Y_test,y_RF)))
print("Accuracy score for RF: {:.4f}".format(accuracy_score(Y_test,y_RF)))
# calculate precision, recall, and f1 scores
prec_rf = precision_score(Y_test,y_RF, average='weighted')
rec_rf = recall_score(Y_test,y_RF,average='weighted')
f1_rf = f1_score(Y_test,y_RF,average='weighted')

Classification report for RF: 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    236110
           1       1.00      1.00      1.00      4123
           2       1.00      1.00      1.00    224498
           3       1.00      1.00      1.00    274797
           4       1.00      1.00      1.00    661850

    accuracy                           1.00   1401378
   macro avg       1.00      1.00      1.00   1401378
weighted avg       1.00      1.00      1.00   1401378

Confusion matrix for RF: 
[[235973      0      0      0    137]
 [     5   4118      0      0      0]
 [     0      0 224498      0      0]
 [     0      0      0 274797      0]
 [     7      0      0      0 661843]]
Accuracy score for RF: 0.9999


# Decision Trees

In [82]:
start = timer.time()
DT_clf= DecisionTreeClassifier()
DT_clf.fit(X_train,Y_train)
end = timer.time()
print("Finished training within {:.2f} seconds".format(end-start))

Finished training within 14.35 seconds


In [83]:
#checking train accuracy
y_DT_train = DT_clf.predict(X_train)

In [85]:
#test accuracy
y_DT = DT_clf.predict(X_test)
y_DT_prob = DT_clf.predict_proba(X_test)

In [86]:
print("Classification report for DT: \n{}".format(classification_report(Y_test,y_DT)))
print("Confusion matrix for DT: \n{}".format(confusion_matrix(Y_test,y_DT)))
print("Accuracy score for DT: {:.4f}".format(accuracy_score(Y_test,y_DT)))
# calculate precision, recall, and f1 scores
prec_dt = precision_score(Y_test,y_DT, average='micro')
rec_dt = recall_score(Y_test,y_DT, average='micro')
f1_dt = f1_score(Y_test,y_DT, average='micro')
print("Precision score for DT: {:.4f}".format(prec_dt))
print("Recall score for DT: {:.4f}".format(rec_dt))
print("F1 score for DF: {:.4f}".format(f1_dt))


Classification report for DT: 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    141721
           1       1.00      1.00      1.00      2445
           2       1.00      1.00      1.00    134804
           3       1.00      1.00      1.00    165272
           4       1.00      1.00      1.00    396585

    accuracy                           1.00    840827
   macro avg       1.00      1.00      1.00    840827
weighted avg       1.00      1.00      1.00    840827

Confusion matrix for DT: 
[[141718      0      0      0      3]
 [     0   2445      0      0      0]
 [     0      0 134804      0      0]
 [     0      0      0 165272      0]
 [     1      0      0      0 396584]]
Accuracy score for DT: 1.0000
Precision score for DT: 1.0000
Recall score for DT: 1.0000
F1 score for DF: 1.0000


# KNN 

In [29]:
start = timer.time()
knn_model = neighbors.KNeighborsClassifier()
knn_model.fit(X_train, Y_train)
end = timer.time()
print("Finished training within {:.2f} seconds".format(end-start))

Finished training within 1.97 seconds


In [30]:
start = timer.time()
y_KNN = knn_model.predict(X_test)
y_KNN_prob = knn_model.predict_proba(X_test)
end = timer.time()
print("Finished training within {:.2f} seconds".format(end-start))

MemoryError: Unable to allocate 1018. MiB for an array with shape (68, 1961928) and data type float64

In [2]:
import time as timer

In [28]:
start = timer.time()
xg_clf=XGBClassifier(use_label_encoder=False)
xg_clf.fit(X_train, Y_train)
end = timer.time()
print("Finished training within {:.2f} seconds".format(end-start))

[21:38:05] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Finished training within 476.36 seconds


In [29]:
#checking train accuracy
y_xg_train = xg_clf.predict(X_train)
print("Accuracy score for DT: {:.4f}".format(accuracy_score(Y_train,y_xg_train)))

Accuracy score for DT: 1.0000


In [30]:
start = timer.time()
y_xg = xg_clf.predict(X_test)
y_xg_prob = xg_clf.predict_proba(X_test)
end = timer.time()
print("Finished training within {:.2f} seconds".format(end-start))

Finished training within 3.46 seconds


In [31]:
print("Classification report for XGBoost: \n{}".format(classification_report(Y_test,y_xg)))
print("Confusion matrix for XGBoost: \n{}".format(confusion_matrix(Y_test,y_xg)))
print("Accuracy score for XGBoost: {:.4f}".format(accuracy_score(Y_test,y_xg)))
# calculate precision, recall, and f1 scores
prec_knn = precision_score(Y_test,y_xg,average='weighted')
rec_knn = recall_score(Y_test,y_xg,average='weighted')
f1_knn = f1_score(Y_test,y_xg,average='weighted')
print("Precision score for XGBoost: {:.4f}".format(prec_knn))
print("Recall score for XGBoost: {:.4f}".format(rec_knn))
print("F1 score for XGBoost: {:.4f}".format(f1_knn))

Classification report for XGBoost: 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    141721
           1       1.00      1.00      1.00      2445
           2       1.00      1.00      1.00    134804
           3       1.00      1.00      1.00    165272
           4       1.00      1.00      1.00    396585

    accuracy                           1.00    840827
   macro avg       1.00      1.00      1.00    840827
weighted avg       1.00      1.00      1.00    840827

Confusion matrix for XGBoost: 
[[141721      0      0      0      0]
 [     0   2445      0      0      0]
 [     0      0 134804      0      0]
 [     0      0      0 165272      0]
 [     0      0      0      0 396585]]
Accuracy score for XGBoost: 1.0000
Precision score for XGBoost: 1.0000
Recall score for XGBoost: 1.0000
F1 score for XGBoost: 1.0000
